In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
import fnmatch
from IPython.display import Markdown as md
import plotly.graph_objects as go
import plotly.express as px
import math
import random
from dask import dataframe as dd
import pickle

In [2]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200
pd.set_option("display.max_colwidth", None)

In [3]:
DATAPATH = Path('../../data/')
DOCSPATH = Path('../../docs/data/')
PLOTSPATH3D = Path('../../docs/plots/3d/')
PLOTSPATH2D = Path('../../docs/plots/2d/')

In [4]:
# Using Dask's DataFrames to glob all the event CSVs in `datapath`

ddf = dd.read_csv(f"{DATAPATH}/overworld/*.csv")["player"].unique().compute()
# Swapping the index/data values so that when `dict(...)` is run we get entries like:
#   "<hash>": <index>,
ddf = pd.DataFrame({"player": ddf.values, "pindex": ddf.index.values})
# ddf.to_csv(DATAPATH / "platerindex.csv", index=False)

playerindex = {player: pindex for (player, pindex) in zip(ddf["player"], ddf["pindex"])}

# NOTE then you can do something like...
#     player2index = dict(pd.read_csv(datapath / "playerindex.csv"))
#   and you'll have a dictionary that does the correct remapping

get_colors = lambda n: list(map(lambda i: "#" + "%06x" % random.randint(0, 0xFFFFFF),range(n)))
colors = get_colors(ddf.player.size)

In [5]:
def get_every_20(df):
    return df.loc[df.index % 20 == 0]

In [6]:
def get_every_30(df):
    return df.loc[df.index % 30 == 0]

In [7]:
def get_every_40(df):
    return df.loc[df.index % 40 == 0]

In [43]:
def stringify_attrs(row, cols: dict) -> str:
    return "\n".join([f"{title}: {row[col]}" for col, title in cols.items()])

def make_metadata(group, event_name):
    if event_name in ["PlayerChangedWorldEvent", "PlayerDeathEvent", "PlayerRespawnEvent", "PlayerEggThrowEvent", "PlayerQuitEvent", "PlayerJoinEvent", "PlayerShearEntityEvent"]:
        return group["day"].apply(lambda day: f"Day {day}")
    elif event_name in ["BlockPlaceEvent", "BlockBreakEvent"]:
        cols = {"day": "Day", "block": "Block"}
        return group[cols].apply(lambda row: stringify_attrs(row, cols), axis=1).values.tolist()
    elif event_name in ["PlayerBucketFillEvent", "PlayerBucketEmptyEvent"]:
        cols = {"day": "Day", "contents": "Bucket Contents"}
        return group[cols].apply(lambda row: stringify_attrs(row, cols), axis=1).values.tolist()
    elif event_name in ["PlayerLeashEntityEvent", "PlayerUnleashEntityEvent"]:
        cols = {"day": "Day", "entity": "Entity"}
        return group[cols].apply(lambda row: stringify_attrs(row, cols), axis=1).values.tolist()
    elif event_name  == "EntityDamageByEntityEvent":
        cols = {"day": "Day", "damaged": "Damaged", "cause": "Cause", "weapon": "Weapon Used"}
        return group[cols].apply(lambda row: stringify_attrs(row, cols), axis=1).values.tolist()
    elif event_name == "PlayerDropItemEvent":
        cols = {"day": "Day", "itemDrop": "Item Dropped"}
        return group[cols].apply(lambda row: stringify_attrs(row, cols), axis=1).values.tolist()
        # day and itemDrop
    elif event_name == "PlayerInteractEvent":
        cols = {"day": "Day", "itemHeld": "Item Held", "action": "Action"}
        return group[cols].apply(lambda row: stringify_attrs(row, cols), axis=1).values.tolist()
    elif event_name == "ProjectileHitEvent":
        cols = {"day": "Day", "weapon": "Weapon Used", "target": "Target"}
        return group[cols].apply(lambda row: stringify_attrs(row, cols), axis=1).values.tolist()

In [29]:
# %% Convert a Player into a Scatter3D trace
def player2trace3d(player, group, event_name):
    scatter = go.Scatter3d(
        mode="markers",
        x=group["x"], y=group["z"], z=group["y"],
        marker={"size": 3, "color": colors[player], "opacity": 0.8},
        name=f"Player {player}",
        text= make_metadata(group, event_name),
    )
    return scatter

In [10]:
# %% Convert a Player into a Scatter3D trace
def player2trace2d(player, group, event_name):
    scatter = go.Scatter(
        mode="markers",
        x=group["x"], y=group["z"],
        marker={"size": 3, "color": colors[player], "opacity": 0.8},
        name=f"Player {player}",
        text= make_metadata(group, event_name),
    )
    return scatter

In [11]:
# %% Convert a DataFrame into a Plot
def df2plot3d(df, name):
    traces = []
    # For every player in the dataframe, convert them into a plotly trace
    for (player, group) in df.groupby("pindex"):
        traces += [player2trace3d(player, group, name)]

    fig = go.Figure(data=traces)
    fig.update_layout({
        "title": {"text": name},
        "width": 900, "height": 900,
    })
    return fig

In [12]:
# %% Convert a DataFrame into a Plot
def df2plot2d(df, name):
    traces = []
    # For every player in the dataframe, convert them into a plotly trace
    for (player, group) in df.groupby("pindex"):
        traces += [player2trace2d(player, group, name)]

    fig = go.Figure(data=traces)
    fig.update_layout({
        "title": {"text": name},
        "width": 900, "height": 900,
    })
    return fig

In [13]:
# %% Preprocess the DataFrame
def preprocess_df(file):
    df = pd.read_csv(file)
    # Use the Player Index created by Dask (above) to do the mappings
    # player2index = dict(pd.read_csv(DATAPATH / "playerindex.csv"))
    df["pindex"] = df["player"].replace(playerindex)

    df = df.rename(columns={
        "location.x": "x", "location.y": "y", "location.z": "z",
        "worldTime": "time",
    })

    df["day"] = np.floor((df["time"] / 1728000)).astype(int)
    # df.to_csv(file.parent / f"{file.stem}-preprocessed.csv", index=False)
    return df

In [14]:
# %% Chunk events by Players
def chunk_by_player(event_path, df, name):
    for (player, group) in df.groupby("pindex"):
        fname = event_path / f"{player}_{name}.csv"
        group.reset_index().to_csv(fname, index=False)

    return df

In [44]:
# %% Preprocess & Plot the Data
def preprocess_and_plot_data_3d(world):
    # NOTE this works iff you're only looking for CSVs.
    for file in (DATAPATH / world).glob("*.csv"):
        name = (os.path.splitext(file.name)[0])
        display(md(f"## {name}"))
        df = preprocess_df(file)

        event_path = DOCSPATH / world / file.stem
        event_path.mkdir(parents=True, exist_ok=True)
        df = chunk_by_player(event_path, df, name)

        # skip_plot = ["PlayerMoveEvent", "EntityDamageByEntityEvent"]
        if file.stem == "PlayerMoveEvent":
            df = get_every_30(df)
        elif file.stem == "EntityDamageByEntityEvent":
            df = get_every_40(df)


        fig = df2plot3d(df, name)
        fig.write_html(str(PLOTSPATH3D / world / name) + ".html", full_html=False, include_plotlyjs="cdn")
        # fig.show()

In [45]:
# %% Preprocess & Plot the Data
def preprocess_and_plot_data_2d(world):
    # NOTE this works iff you're only looking for CSVs.
    for file in (DATAPATH / world).glob("*.csv"):
        name = (os.path.splitext(file.name)[0])
        display(md(f"## {name}"))
        df = preprocess_df(file)

        event_path = DOCSPATH / world / file.stem
        event_path.mkdir(parents=True, exist_ok=True)
        df = chunk_by_player(event_path, df, name)

        # skip_plot = ["PlayerMoveEvent", "EntityDamageByEntityEvent"]
        if file.stem == "PlayerMoveEvent":
            df = get_every_40(df)
        elif file.stem == "EntityDamageByEntityEvent":
            df = get_every_40(df)


        fig = df2plot2d(df, name)
        fig.write_html(str(PLOTSPATH2D / world / name) + ".html", full_html=False, include_plotlyjs="cdn")
        # fig.show()

# 3D Plots

## Overworld

In [46]:
preprocess_and_plot_data_3d("overworld")

## ProjectileHitEvent

## PlayerQuitEvent

## PlayerBedLeaveEvent

## PlayerBedEnterEvent

## PlayerBucketEmptyEvent

...

## BlockPlaceEvent

## BlockBreakEvent

## PlayerJoinEvent

## PlayerShearEntityEvent

## EnchantItemEvent

## Nether

In [47]:
preprocess_and_plot_data_3d("nether")

## ProjectileHitEvent

## PlayerQuitEvent

## PlayerBedLeaveEvent

## PlayerBedEnterEvent

## PlayerBucketEmptyEvent

...

## BlockPlaceEvent

## BlockBreakEvent

## PlayerJoinEvent

## PlayerShearEntityEvent

## EnchantItemEvent

## End

In [48]:
# random_color = make_random_color()
preprocess_and_plot_data_3d("end")

## ProjectileHitEvent

## PlayerQuitEvent

## PlayerBedLeaveEvent

## PlayerBedEnterEvent

## PlayerBucketEmptyEvent

...

## BlockPlaceEvent

## BlockBreakEvent

## PlayerJoinEvent

## PlayerShearEntityEvent

## EnchantItemEvent

# 2D Plots

## Overworld

In [49]:
preprocess_and_plot_data_2d("overworld")

## ProjectileHitEvent

## PlayerQuitEvent

## PlayerBedLeaveEvent

## PlayerBedEnterEvent

## PlayerBucketEmptyEvent

...

## BlockPlaceEvent

## BlockBreakEvent

## PlayerJoinEvent

## PlayerShearEntityEvent

## EnchantItemEvent

## Nether

In [50]:
preprocess_and_plot_data_2d("nether")

## ProjectileHitEvent

## PlayerQuitEvent

## PlayerBedLeaveEvent

## PlayerBedEnterEvent

## PlayerBucketEmptyEvent

...

## BlockPlaceEvent

## BlockBreakEvent

## PlayerJoinEvent

## PlayerShearEntityEvent

## EnchantItemEvent

## End

In [51]:
preprocess_and_plot_data_2d("end")

## ProjectileHitEvent

## PlayerQuitEvent

## PlayerBedLeaveEvent

## PlayerBedEnterEvent

## PlayerBucketEmptyEvent

...

## BlockPlaceEvent

## BlockBreakEvent

## PlayerJoinEvent

## PlayerShearEntityEvent

## EnchantItemEvent

# Qualtrics Pre-Survey

In [ ]:
df_q = pd.read_csv("../../data/01-qualtrics.csv")
df_q = df_q.drop(columns=['StartDate',	'EndDate',	'Status',	'IPAddress',	'Progress',	'Duration (in seconds)',	'Finished',	'RecordedDate',	'ResponseId',	'RecipientLastName',	'RecipientFirstName',	'RecipientEmail',	'ExternalReference',	'LocationLatitude',	'LocationLongitude',	'DistributionChannel',	'UserLanguage', 'mc_dim_4_TEXT'])	

In [ ]:
df_q = df_q.drop(index=[0, 1, 2, 3, 47, 48, 49, 50, 51, 52, 53, 54,55, 58, 59, 60, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 90, 106]).reset_index(drop=True)
df_q.to_csv("../../data/02-qualtrics.csv", index=False)

In [ ]:
df_q = df_q.drop(columns=["username", "age_check", "agree_rules", "consent"]).reset_index(drop=True)
df_q = df_q.rename(columns={"age_1": "Age","expertise_1": "overall_exp", "expertise_2":"single_exp", "expertise_3":"multi_exp"})

In [ ]:
df_act = pd.DataFrame(df_q["mc_act"].str.get_dummies(sep=",").T.sum(axis=1))
df_act = df_act.rename(columns={0:"count"})
df_act

In [ ]:
df_mode = pd.DataFrame(df_q["mc_mode"].str.get_dummies(sep=",").T.sum(axis=1))
df_mode = df_mode.rename(columns={0:"count"})
df_mode

In [ ]:
df_lvl = pd.DataFrame(df_q["mc_level"].str.get_dummies(sep=",").T.sum(axis=1))
df_lvl = df_lvl.rename(columns={0:"count"})
df_lvl

In [ ]:
df_ref = pd.DataFrame(df_q["study_ref"].str.get_dummies(sep=",").T.sum(axis=1))
df_ref = df_ref.rename(columns={0:"count"})
df_ref

In [ ]:
fig = px.bar(df_act.sort_values(by="count"), x="count", title="Minecraft Activities")
fig.write_html(str(PLOTSPATH2D) + "/mc_act.html", full_html=False, include_plotlyjs="cdn")
fig.show()

In [ ]:
fig = px.bar(df_mode.sort_values(by="count"), x="count", title="Minecraft Player Mode")
fig.write_html(str(PLOTSPATH2D) + "/mc_mode.html", full_html=False, include_plotlyjs="cdn")
fig.show()

In [ ]:
fig = px.bar(df_lvl.sort_values(by="count"), x="count", title="Minecraft Player Level")
fig.write_html(str(PLOTSPATH2D) + "/mc_level.html", full_html=False, include_plotlyjs="cdn")
fig.show()

In [ ]:
fig = px.bar(df_ref.sort_values(by="count"), x="count", title="Minecraft Study Reference")
fig.write_html(str(PLOTSPATH2D) + "/mc_study_ref.html", full_html=False, include_plotlyjs="cdn")
fig.show()

In [ ]:
fig = px.bar(df_q, x="Age", title="Player Age")
fig.write_html(str(PLOTSPATH2D) + "/player_age.html", full_html=False, include_plotlyjs="cdn")
fig.show()

In [ ]:
df_exp = df_q[["overall_exp", "single_exp", "multi_exp"]].melt(var_name="type", value_name="level")
df_exp = df_exp.replace({"overall_exp": "Overall", "single_exp": "Singleplayer", "multi_exp": "Multiplayer"})

fig = px.bar(df_exp.sort_values(by="level"), x="level", barmode="group",color="type",title="Player Experience")
fig.write_html(str(PLOTSPATH2D) + "/player_exp.html", full_html=False, include_plotlyjs="cdn")
fig.show()

In [ ]:
df_q.to_csv("../../public/data/01-qualtrics.csv", index=False)

# Old Code

In [ ]:
# def list_to_dic(lst):
#     index = [i for i in range(len(lst))]
#     zipped = list(zip(lst,index))
#     ret = {zipped[i][0]: zipped[i][1] for i in range(len(zipped))}
#     return ret

In [ ]:
# def process_data(world):
#     basepath = '../../data/'
#     datapath = '../../docs/data/'
    
#     with os.scandir(basepath + world) as entries:
#         for entry in entries:
#             if entry.is_file():
#                 event_file = entry.name
#                 event_name = (os.path.splitext(entry.name)[0])

#                 event_df = pd.read_csv(basepath + world + event_file)
#                 event_df.drop(columns=['event'], inplace=True)
                
#                 unique_players = list(event_df['player'].unique())
#                 player_mapping = list_to_dic(unique_players)
#                 event_df['player'] = event_df['player'].map(player_mapping)
#                 unique_players = list(event_df['player'].unique())

#                 event_df.rename(columns={'location.x': 'x', 'location.y': 'y', 'location.z': 'z', 'worldTime': 'time'}, inplace=True)
#                 event_df['mc_day'] = (event_df['time'] / 24000).astype(int)
#                 display(md(f'## {event_name}'))
#                 # display(event_df)

#                 path = Path(datapath + world + event_name + '/')
#                 path.mkdir(exist_ok=True)
                

#                 if fnmatch.fnmatch(event_file, 'PlayerMoveEvent.csv') or fnmatch.fnmatch(event_file, 'EntityDamageByEntityEvent.csv'):
#                     make_csv_large(event_df, unique_players, datapath + world + event_name + '/', event_name)
#                 else:
#                     make_csv_small(event_df, unique_players, datapath + world + event_name + '/', event_name)
#                     # players_ls = make_ls_players(event_df, unique_players)
#                     # plot = make_player_plots(players_ls, color)
#                     # plot.show()
                

In [ ]:
# def make_csv_large(event_df, players, path, event_name):
#     for player in players:
#         df = event_df[event_df['player'] == player]

#         df = df[df.index % 20 == 0]

#         filename = path + str(player) + '_' + event_name + '.csv'
#         df.to_csv(filename, index=False)

In [ ]:
# def make_csv_small(event_df, players, path, event_name):
#     for player in players:
#         df = event_df[event_df['player'] == player]

#         filename = path + str(player) + '_' + event_name + '.csv'
#         df.to_csv(filename, index=False)

In [ ]:
# def make_ls_players(event_df, players):
#     players_ls = []

#     for player in players:
#         df = event_df[event_df['player'] == player]
#         players_ls.append(df)
        
#     return players_ls

In [ ]:
# def make_player_plots(players, random_color):
#     data = []
    
#     for df in players:
#         scatter = go.Scatter3d(
#             mode='markers',
#             x=df["x"], y=df["z"], z=df["y"],
#             name = 'Player' + df,
#             marker={
#                 "size": 3,
#                 "color": random_color, 
#                 # "colorscale": "Rainbow",
#                 "opacity": 0.8
#             },
#             # line=dict(
#             #     color='darkblue',
#             #     width=2
#             # ),
#             # text = [df for _ in range(len(df))]
#         )
#         data.append(scatter)
#     fig = go.Figure(data=data)
#     fig.update_layout(width=600, height=400)
# #     fig.show()
#     return fig

In [ ]:
# def make_random_color():
#     random_color = []
    
#     for i in range(0, 40): 
#         r = str(math.floor(random.randrange(0, 256)))
#         g = str(math.floor(random.randrange(0, 256)))
#         b = str(math.floor(random.randrange(0, 256)))
#         random_color.append('rgb(' + r + ',' + g + ',' + b +')')
        
#     return random_color

In [ ]:
# import plotly.graph_objects as go
# import plotly.express as px

In [ ]:
# def get_data(world, event):
#     event_file = '../../data/' + world + '/' + event + '.csv'
#     event_df = pd.read_csv(event_file)
#     display(event_df)
    
#     # drop unnecessary columns
#     event_df.drop(columns=['_id','event'], inplace=True)
#     display(event_df)
    
#     unique_players = list(event_df['player'].unique())
#     print(unique_players)
    
#     player_mapping = list_to_dic(unique_players)
    
#     event_df['player'] = event_df['player'].map(player_mapping)
    
#     unique_players = list(event_df['player'].unique())
#     print(unique_players)
    
#     event_df.rename(columns={'location.x': 'x', 'location.y': 'y', 'location.z': 'z', 'worldTime': 'time'}, inplace=True)
    
#     return (event_df, unique_players)

# def make_csv_plots(move, players, world, folder):
#     for player in players:
#         # df = move[move['player'] == player]
#         # print('BEFORE: ' + str(len(df)))
#         #drop all odd numbered rows
#     #     drop_i = [i for i in list(range(len(df))) if i % 2 == 1]
#     #     df.drop(drop_i, inplace=True)
#         df = move[move.index % 20 == 0]
#         # print('AFTER: ', str(len(df)))

#     filepath = '../../docs/data/' + world + "/" + folder + "/"
#     filename = filepath + 'plot' + '_' + folder + '.csv'
#     df.to_csv(filename, index=False)
#     # return df
    

In [ ]:
# (df, unique_players) = get_data('overworld', 'PlayerMoveEvent')
# make_csv_plots(df, unique_players, 'overworld', 'move')